In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# data=pd.read_csv('C:/Users/ansh41/Desktop/mypython/codingNinja/datasets/training_twitter_x_y_train.csv')

In [ ]:
# test_data=pd.read_csv('C:/Users/ansh41/Desktop/mypython/codingNinja/datasets/test_twitter_x_test.csv')

In [ ]:
data=pd.read_csv("twitter_train.csv",delimiter=",")
test_data=pd.read_csv("twitter_test.csv",delimiter=",")

In [ ]:
data.head()

In [ ]:
y_train=data['airline_sentiment']
x_train=data['text']

In [ ]:
x_train.head(),y_train.head()

In [ ]:
x_test=test_data['text']

In [ ]:
data['airline'].unique()

In [ ]:
airline=['@SouthwestAir', '@united', '@JetBlue', '@USAirways', '@AmericanAir',
       '@VirginAmerica','']
tag=set(airline)

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.corpus import wordnet
import nltk

In [ ]:
stops=set(stopwords.words('english'))
punct=list(string.punctuation)
stops.update(punct)
stops.update(tag)
len(stops)

In [ ]:
all_tweets=[''.join(tweet) for tweet in x_train]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer(max_features=1200,ngram_range=(1,3),stop_words=stops)

In [ ]:
x_train_features=cv.fit_transform(x_train)

In [ ]:
x_train_features.todense()

In [ ]:
cv.get_feature_names()

In [ ]:
x_test_features=cv.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc=RandomForestClassifier()

In [ ]:
rfc.fit(x_train_features,y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
DTC=DecisionTreeClassifier()
DTC.fit(x_train_features,y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knc=KNeighborsClassifier()
knc.fit(x_train_features,y_train)

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb=GaussianNB()
gnb.fit(x_train_features.toarray(),y_train)

In [ ]:
y_pred=rfc.predict(x_test_features)

In [ ]:
y_pred=DTC.predict(x_test_features)

In [ ]:
y_pred=knc.predict(x_test_features)

In [ ]:
y_pred=gnb.predict(x_test_features.toarray())

In [ ]:
np.savetxt('ff.csv',y_pred,fmt='%s')

In [ ]:
documents=[ (tweet.split(" "),sentiment) for tweet,sentiment in zip(x_train,y_train)]
test_documents=[tweet.split(" ") for tweet in x_test]

In [ ]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
def clean_review(words):
    output_words=[]
    for word in words:
        if word.lower() not in stops:
            
            pos=pos_tag([word])
            clean_word=lemmatizer.lemmatize(word,get_simple_pos(pos[0][1]))
            output_words.append(clean_word)
    return output_words

In [ ]:
documents=[(clean_review(document),category) for document,category in documents]
test_documents=[clean_review(document) for document in test_documents]
len(test_documents)

In [ ]:
all_words=[]
for document in documents:
    all_words+=document[0]

In [ ]:
dictionary= nltk.FreqDist(all_words)

In [ ]:
common=dictionary.most_common(2000)
features=[i[0] for i in common]

In [ ]:
def get_feature_dict(words):
    current_features={}
    word_set=set(words)
    for w in features:
        current_features[w]=w in word_set
    return current_features

In [ ]:
training_data=[(get_feature_dict(doc),category) for doc,category in documents]

In [ ]:
testing_data=[get_feature_dict(doc) for doc in test_documents]

In [ ]:
from nltk import NaiveBayesClassifier

In [ ]:
clf=NaiveBayesClassifier.train(training_data)

In [ ]:
clf.show_most_informative_features(15)

In [ ]:
y_pred=[]
for i in range(len(testing_data)):
    y_pred.append(clf.classify(testing_data[i]))


In [ ]:
np.savetxt('khushi.csv',y_pred,fmt='%s')